In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 데이터 불러오기

In [4]:
import pandas as pd

## 레시피데이터 불러오기

In [5]:
df = pd.read_csv('/content/drive/MyDrive/MyDrive/new_re2.csv', encoding='utf-8')

In [6]:
columns_to_drop = ['원자재코드', '총중량','투입지시중량','제품여부']

df2=df.drop(columns_to_drop, axis=1)

## 생산계획모델 불러오기

In [7]:
df3 = pd.read_csv('/content/drive/MyDrive/MyDrive/allpbbc.csv', encoding='utf-8')
df3.loc[df3['생산량'] <= 0, '생산량'] = 0

In [8]:
df3

,week,prodname,생산량
0,2021-04-26,AE,10867.0
1,2021-04-26,PR1000,14037.0
2,2021-04-26,CSA5000,252725.0
3,2021-04-26,PEMA-SR5000F,1.0
4,2021-04-26,PEMA-500FR,41476.0
...,...,...,...
373,2021-07-26,CSA4000,19145.0
374,2021-07-26,PEMA-HR1000S,1.0
375,2021-07-26,PEMA-PCR3000N,0.0
376,2021-07-26,PEMA-CSA5000,0.0


## 원자재 사용량으로 변환

In [9]:
result_dict = {}

# 각 날짜별로 레시피에 따라 원자재의 양을 계산하여 저장
for idx, row in df3.iterrows(): # idx 인덱스의 값을 추출 , row 객체의 시리즈 내용 값을 추출
    week = row['week']
    prodname = row['prodname']
    생산량 = row['생산량']

    if week not in result_dict:
        result_dict[week] = {}

    # 해당 제품의 레시피 비율을 가져옴
    recipe =df2[df2['제품명'] == prodname]

    # 레시피에 따라 원자재의 양 계산
    for r_idx, r_row in recipe.iterrows():
        원자재명 = r_row['원자재명']
        비율 = r_row['비율']

        원자재_생산량 = 생산량 * 비율

        if 원자재명 not in result_dict[week]:
            result_dict[week][원자재명] = 0

        result_dict[week][원자재명] += 원자재_생산량

print(result_dict)

{'2021-04-26': {'사용수': 667623.7409780001, '공기연행제': 5979.944767999999, 'RA-300S': 23173.184024999995, 'PEMA-WRE-600(5': 11047.236922999999, 'PC소포제': 386.073364, 'PEMA-HR1000 태명': 666.1831, 'PEMA-SR3000F 현': 541.0700770000001, 'PEMA-HR1000 영동': 767.0022220000001, 'PEMA-PR2000 영풍': 112.968075, 'PEMA-SR3000F 대': 209.25629999999998, 'PEMA-HR1500 삼덕': 774.28408, 'CSA5000 서원(태안)': 866.3413, 'CSA5000 삼표(풍납)': 461.466418, 'PEMA-HR1500 신송': 589.0676159999999, 'PEMA-SR2000 정선': 653.836157, 'PEMA-SR2000 울산': 333.230214, '액상리그닌': 1748.57217, '글루콘산소다': 11376.319952, 'ACTICIDE-MVK': 397.81297400000005, 'PEMA-SRE-200(5': 10364.518539, 'WRE-580FX(55%)': 115524.09244599998, 'PEMA-WRE-550(5': 9343.795725, 'WRE-770': 87358.67112099999, 'WRE-550T': 48434.239694, 'SRE-200SD-T': 15273.978215000001, 'PEMA-500FR': 11683.430583, 'PEMA-630XR': 8487.321292, 'M30': 10888.860785, 'PCA3000': 23799.898407, 'Urea-20%': 66682.19465599998, '소포제(MR-130)': 793.852577, 'AE 산하(덕소)': 165.871452, 'PEMA-550XD': 4231.207222, 'S

In [10]:
result_data = []
for week, material_dict in result_dict.items():
    row = {'week': week}
    row.update(material_dict)
    result_data.append(row)

result_df = pd.DataFrame(result_data)
result_df.set_index('week', inplace=True)
result_df

,사용수,공기연행제,RA-300S,PEMA-WRE-600(5,PC소포제,PEMA-HR1000 태명,PEMA-SR3000F 현,PEMA-HR1000 영동,PEMA-PR2000 영풍,PEMA-SR3000F 대,...,PEMA-SR3000F 극,PEMA-HR1500 우성,PEMA-SN400 하남(,PEMA-SN400 청명(,PEMA-SN400 유익(,PEMA-SR2000 영남,PEMA-SP1000 대보,MONODN1501,MONODN1502,BDG
week,,,,,,,,,,,,,,,,,,,,,
2021-04-26,667623.740978,5979.944768,23173.184025,11047.236923,386.073364,666.183100,541.070077,767.002222,112.968075,209.256300,...,538.027679,488.354101,389.176164,1614.764970,460.201014,529.527008,225.116176,11.151,11.151,163.548
2021-05-03,835098.035629,14420.388354,34880.458923,16360.770096,424.223202,643.589944,647.620663,982.095946,109.136838,202.159512,...,815.027758,795.726590,32.407032,134.462860,38.321332,760.786211,292.491715,9.750,9.750,143.000
2021-05-10,881587.523432,13574.549505,36011.483209,4727.594329,432.903809,816.804140,682.340717,1133.087772,138.509655,256.568220,...,950.270705,745.426147,159.759276,662.871230,188.915426,1021.842990,645.585846,9.450,9.450,138.600
2021-05-17,876730.601959,7181.411292,35838.075597,4901.622572,439.479945,564.014376,606.957012,1160.410059,95.642802,177.163848,...,1159.623585,939.304143,211.384884,877.075570,249.962734,1225.038915,524.845335,19.686,19.686,288.728
2021-05-24,852179.201331,4344.202860,33369.424653,5904.976053,442.660664,670.506140,645.614729,1046.179844,113.701155,210.614220,...,905.011917,803.022711,389.176164,1614.764970,460.201014,904.342054,334.315574,10.203,10.203,149.644
2021-05-31,759963.982408,6886.827640,28316.155580,3681.065626,400.031189,628.356500,563.163956,924.786151,106.553625,197.374500,...,788.820824,659.700520,288.453708,1196.848590,341.096658,817.977608,501.032704,0.003,0.003,0.044
2021-06-07,819858.014779,14447.780170,30310.760909,5001.762041,441.414745,635.304996,571.667229,1043.931751,107.731917,199.557108,...,967.520702,738.207862,288.453708,1196.848590,341.096658,1055.733689,475.519597,29.220,29.220,428.560
2021-06-14,667177.718606,7360.534652,27523.535473,2908.957813,335.491897,401.821296,419.552673,905.728365,68.138892,126.217008,...,959.272791,700.715325,211.384884,877.075570,249.962734,1069.801272,415.091244,0.000,0.000,0.000
2021-06-21,630594.468254,6688.501169,21346.857684,10279.502912,378.098677,588.885036,489.529542,675.149199,99.860247,184.976028,...,463.503338,445.499806,389.176164,1614.764970,460.201014,437.851836,215.363616,13.866,13.866,203.368


In [ ]:
result_df.to_csv(f'/content/drive/MyDrive/result_df.csv')

## 기초재고설정

In [11]:
row_count = len(result_df)
col_count = result_df.shape[1]
x=result_df.columns
x=[x]
re={}
for i in x[0]:
  re[i]=result_df[i][0]+result_df[i][1]


In [12]:
re

{'사용수': 1502721.7766070003,
 '공기연행제': 20400.333122,
 'RA-300S': 58053.642947999986,
 'PEMA-WRE-600(5': 27408.007018999997,
 'PC소포제': 810.296566,
 'PEMA-HR1000 태명': 1309.773044,
 'PEMA-SR3000F 현': 1188.69074,
 'PEMA-HR1000 영동': 1749.098168,
 'PEMA-PR2000 영풍': 222.104913,
 'PEMA-SR3000F 대': 411.41581199999996,
 'PEMA-HR1500 삼덕': 1771.892089,
 'CSA5000 서원(태안)': 1703.301212,
 'CSA5000 삼표(풍납)': 990.549588,
 'PEMA-HR1500 신송': 1110.829385,
 'PEMA-SR2000 정선': 1191.9371099999998,
 'PEMA-SR2000 울산': 739.298132,
 '액상리그닌': 4204.053583999999,
 '글루콘산소다': 26634.073201,
 'ACTICIDE-MVK': 901.129702,
 'PEMA-SRE-200(5': 22828.902025,
 'WRE-580FX(55%)': 305582.79264,
 'PEMA-WRE-550(5': 19705.072578,
 'WRE-770': 206534.97414599996,
 'WRE-550T': 105434.54669300001,
 'SRE-200SD-T': 34679.908356,
 'PEMA-500FR': 17984.811029999997,
 'PEMA-630XR': 14457.619015,
 'M30': 22371.612717,
 'PCA3000': 56239.29075099999,
 'Urea-20%': 155805.158154,
 '소포제(MR-130)': 1847.5017579999999,
 'AE 산하(덕소)': 380.175692,
 'PEMA-55

In [13]:
col_count

54

In [14]:
inven = pd.DataFrame([re], columns=re.keys())

inven

,사용수,공기연행제,RA-300S,PEMA-WRE-600(5,PC소포제,PEMA-HR1000 태명,PEMA-SR3000F 현,PEMA-HR1000 영동,PEMA-PR2000 영풍,PEMA-SR3000F 대,...,PEMA-SR3000F 극,PEMA-HR1500 우성,PEMA-SN400 하남(,PEMA-SN400 청명(,PEMA-SN400 유익(,PEMA-SR2000 영남,PEMA-SP1000 대보,MONODN1501,MONODN1502,BDG
0,1.502722e+06,20400.333122,58053.642948,27408.007019,810.296566,1309.773044,1188.69074,1749.098168,222.104913,411.415812,...,1353.055437,1284.080691,421.583196,1749.22783,498.522346,1290.313219,517.607891,20.901,20.901,306.548


## 안전재고량 설정

In [15]:
larger_values = []
mean_values = []
safs = []

for i in range(row_count - 1):
    for col_index in range(col_count):  # 열 인덱스 범위로 순회합니다
        col_name = result_df.columns[col_index]  # 실제 열 이름을 가져옵니다
        larger_value = max(result_df[col_name][i], result_df[col_name][i+1])
        mean_value = (result_df[col_name][i] + result_df[col_name][i+1]) / 2
        saf = (larger_value * 3) - (mean_value * 3)

        larger_values.append(larger_value)
        mean_values.append(mean_value)
        safs.append(saf)


In [16]:
# 안전재고량 뽑기
import pandas as pd

# 새로운 데이터프레임 생성
num_columns = col_count
data = {}  # 열 데이터를 저장할 딕셔너리

for i in range(num_columns):
    column_name = f'column_{i+1}'
    data[column_name] = []  # 빈 리스트로 초기화


# safs 리스트의 값을 데이터프레임 열에 할당
for i, value in enumerate(safs):
    column_name = f'column_{(i % num_columns) + 1}'  # 열 인덱스를 순환시킴
    data[column_name].append(value)

safe_inven = pd.DataFrame(data)
safe_inven.columns = result_df.columns
extracted_indices = result_df.index[:13]
safe_inven.index = extracted_indices
safe_inven


,사용수,공기연행제,RA-300S,PEMA-WRE-600(5,PC소포제,PEMA-HR1000 태명,PEMA-SR3000F 현,PEMA-HR1000 영동,PEMA-PR2000 영풍,PEMA-SR3000F 대,...,PEMA-SR3000F 극,PEMA-HR1500 우성,PEMA-SN400 하남(,PEMA-SN400 청명(,PEMA-SN400 유익(,PEMA-SR2000 영남,PEMA-SP1000 대보,MONODN1501,MONODN1502,BDG
week,,,,,,,,,,,,,,,,,,,,,
2021-04-26,251211.441976,12660.665379,17560.912347,7970.299760,57.224757,33.889734,159.825879,322.640586,5.746855,10.645182,...,415.500119,461.058733,535.153698,2220.453165,632.819523,346.888804,101.063309,2.1015,2.1015,30.822
2021-05-03,69734.231705,1268.758274,1696.536429,17449.763651,13.020911,259.821294,52.080081,226.487739,44.059225,81.613062,...,202.864421,75.450665,191.028366,792.612555,225.891141,391.585168,529.641196,0.4500,0.4500,6.600
2021-05-10,7285.382210,9589.707319,260.111418,261.042364,9.864204,379.184646,113.075557,40.983431,64.300279,119.106558,...,314.029320,290.816994,77.438412,321.306510,91.570962,304.793887,181.110766,15.3540,15.3540,225.192
2021-05-17,36827.100942,4255.812648,3702.976416,1505.030222,4.771079,159.737646,57.986575,171.345323,27.087530,50.175558,...,381.917502,204.422148,266.686920,1106.534100,315.357420,481.045291,285.794642,14.2245,14.2245,208.626
2021-05-24,138322.828384,3813.937170,7579.903610,3335.865641,63.944213,63.224460,123.676159,182.090539,10.721295,19.859580,...,174.286640,214.983287,151.083684,626.874570,178.656534,129.546669,250.075695,15.3000,15.3000,224.400
2021-05-31,89841.048556,11341.428795,2991.907994,1981.044622,62.075334,10.422744,12.754909,178.718400,1.767438,3.273912,...,268.049817,117.761013,0.000000,0.000000,0.000000,356.634122,38.269661,43.8255,43.8255,642.774
2021-06-07,229020.444260,10630.868277,4180.838154,3139.206342,158.884272,350.225550,228.171834,207.305079,59.389538,110.010150,...,12.371867,56.238806,115.603236,479.659530,136.700886,21.101375,90.642529,43.8300,43.8300,642.840
2021-06-14,54874.875528,1008.050224,9265.016683,11055.817648,63.910170,280.595610,104.965303,345.868749,47.582032,88.138530,...,743.654180,382.823278,266.686920,1106.534100,315.357420,947.924154,299.591442,20.7990,20.7990,305.052
2021-06-21,248158.551739,10837.040381,18110.009267,11577.541635,71.256431,27.583104,145.622517,351.520299,4.677408,8.664192,...,467.699049,452.377467,514.612386,2135.223405,608.529411,439.711938,128.106594,0.0000,0.0000,0.000


## 안전재고량이랑 기초재고량 합산 테이블

In [17]:
amt = []

# result_df.columns를 기준으로 루프 실행
for i in result_df.columns:
    # inven의 첫 번째 행 값
    inven_value = inven[i][0]

    # safe_inven의 각 행 값 더하기
    for j in range(len(safe_inven)):
        safe_inven_value = safe_inven[i][j]
        total_value = inven_value + safe_inven_value
        amt.append(total_value)

In [18]:
num_elements_per_row = row_count-1
result_matrix = []

while amt:
    row = amt[:num_elements_per_row]
    amt = amt[num_elements_per_row:]
    result_matrix.append(row)

for row in result_matrix:
    row

In [19]:
result_matrix2 = pd.DataFrame(result_matrix)
amt_df = result_matrix2.T
amt_df.columns = result_df.columns
extracted_indices = result_df.index[1:14]
amt_df.index = extracted_indices
amt_df # 안전재고 +재고


,사용수,공기연행제,RA-300S,PEMA-WRE-600(5,PC소포제,PEMA-HR1000 태명,PEMA-SR3000F 현,PEMA-HR1000 영동,PEMA-PR2000 영풍,PEMA-SR3000F 대,...,PEMA-SR3000F 극,PEMA-HR1500 우성,PEMA-SN400 하남(,PEMA-SN400 청명(,PEMA-SN400 유익(,PEMA-SR2000 영남,PEMA-SP1000 대보,MONODN1501,MONODN1502,BDG
week,,,,,,,,,,,,,,,,,,,,,
2021-05-03,1.753933e+06,33060.998501,75614.555295,35378.306779,867.521323,1343.662778,1348.516619,2071.738754,227.851768,422.060994,...,1768.555556,1745.139424,956.736894,3969.680995,1131.341869,1637.202023,618.671200,23.0025,23.0025,337.370
2021-05-10,1.572456e+06,21669.091396,59750.179377,44857.770669,823.317477,1569.594338,1240.770821,1975.585907,266.164138,493.028874,...,1555.919858,1359.531356,612.611562,2541.840385,724.413487,1681.898387,1047.249087,21.3510,21.3510,313.148
2021-05-17,1.510007e+06,29990.040441,58313.754366,27669.049383,820.160770,1688.957690,1301.766297,1790.081599,286.405192,530.522370,...,1667.084757,1574.897685,499.021608,2070.534340,590.093308,1595.107106,698.718657,36.2550,36.2550,531.740
2021-05-24,1.539549e+06,24656.145770,61756.619364,28913.037241,815.067645,1469.510690,1246.677315,1920.443491,249.192443,461.591370,...,1734.972939,1488.502839,688.270116,2855.761930,813.879766,1771.358510,803.402533,35.1255,35.1255,515.174
2021-05-31,1.641045e+06,24214.270292,65633.546557,30743.872659,874.240779,1372.997504,1312.366899,1931.188707,232.826208,431.275392,...,1527.342077,1499.063978,572.666880,2376.102400,677.178880,1419.859888,767.683586,36.2010,36.2010,530.948
2021-06-07,1.592563e+06,31741.761917,61045.550941,29389.051641,872.371900,1320.195788,1201.445649,1927.816568,223.872351,414.689724,...,1621.105254,1401.841704,421.583196,1749.227830,498.522346,1646.947341,555.877552,64.7265,64.7265,949.322
2021-06-14,1.731742e+06,31031.201399,62234.481102,30547.213361,969.180838,1659.998594,1416.862574,1956.403247,281.494451,521.425962,...,1365.427304,1340.319497,537.186432,2228.887360,635.223232,1311.414593,608.250420,64.7310,64.7310,949.388
2021-06-21,1.557597e+06,21408.383346,67318.659631,38463.824667,874.206736,1590.368654,1293.656043,2094.966917,269.686945,499.554342,...,2096.709617,1666.903969,688.270116,2855.761930,813.879766,2238.237373,817.199333,41.7000,41.7000,611.600
2021-06-28,1.750880e+06,31237.373503,76163.652215,38985.548654,881.552997,1337.356148,1334.313257,2100.618467,226.782321,420.080004,...,1820.754486,1736.458158,936.195582,3884.451235,1107.051757,1730.025157,645.714485,20.9010,20.9010,306.548


## 매주 원자재별 발주량

In [20]:
result_df_aligned = result_df.reindex(index=safe_inven.index, columns=safe_inven.columns, fill_value=0)
raw_order = safe_inven + result_df_aligned

In [21]:
raw_order # 필요한 발주량

,사용수,공기연행제,RA-300S,PEMA-WRE-600(5,PC소포제,PEMA-HR1000 태명,PEMA-SR3000F 현,PEMA-HR1000 영동,PEMA-PR2000 영풍,PEMA-SR3000F 대,...,PEMA-SR3000F 극,PEMA-HR1500 우성,PEMA-SN400 하남(,PEMA-SN400 청명(,PEMA-SN400 유익(,PEMA-SR2000 영남,PEMA-SP1000 대보,MONODN1501,MONODN1502,BDG
week,,,,,,,,,,,,,,,,,,,,,
2021-04-26,9.188352e+05,18640.610147,40734.096372,19017.536683,443.298121,700.072834,700.895956,1089.642808,118.714930,219.901482,...,953.527798,949.412834,924.329862,3835.218135,1093.020537,876.415812,326.179485,13.2525,13.2525,194.370
2021-05-03,9.048323e+05,15689.146628,36576.995352,33810.533747,437.244113,903.411238,699.700744,1208.583685,153.196063,283.772574,...,1017.892179,871.177255,223.435398,927.075415,264.212473,1152.371379,822.132911,10.2000,10.2000,149.600
2021-05-10,8.888729e+05,23164.256824,36271.594627,4988.636693,442.768013,1195.988786,795.416274,1174.071203,202.809934,375.674778,...,1264.300025,1036.243141,237.197688,984.177740,280.486388,1326.636877,826.696612,24.8040,24.8040,363.792
2021-05-17,9.135577e+05,11437.223940,39541.052013,6406.652794,444.251024,723.752022,664.943587,1331.755382,122.730332,227.339406,...,1541.541087,1143.726291,478.071804,1983.609670,565.320154,1706.084206,810.639977,33.9105,33.9105,497.354
2021-05-24,9.905020e+05,8158.140030,40949.328263,9240.841694,506.604877,733.730600,769.290888,1228.270383,124.422450,230.473800,...,1079.298557,1018.005998,540.259848,2241.639540,638.857548,1033.888723,584.391269,25.5030,25.5030,374.044
2021-05-31,8.498050e+05,18228.256435,31308.063574,5662.110249,462.106523,638.779244,575.918866,1103.504551,108.321063,200.648412,...,1056.870641,777.461533,288.453708,1196.848590,341.096658,1174.611730,539.302365,43.8285,43.8285,642.818
2021-06-07,1.048878e+06,25078.648447,34491.599063,8140.968383,600.299017,985.530546,799.839063,1251.236830,167.121455,309.567258,...,979.892569,794.446668,404.056944,1676.508120,477.797544,1076.835063,566.162126,73.0500,73.0500,1071.400
2021-06-14,7.220526e+05,8368.584876,36788.552156,13964.775461,399.402067,682.416906,524.517976,1251.597114,115.720924,214.355538,...,1702.926971,1083.538603,478.071804,1983.609670,565.320154,2017.725426,714.682686,20.7990,20.7990,305.052
2021-06-21,8.787530e+05,17525.541550,39456.866951,21857.044547,449.355108,616.468140,635.152059,1026.669498,104.537655,193.640220,...,931.202387,897.877273,903.788550,3749.988375,1068.730425,877.563774,343.470210,13.8660,13.8660,203.368
